# This is a heading. 
## this is a subheading. 
**this is bold**
- this is a list item

*this is italic*
<br>
$this is math text.$




In [9]:
import numpy as np 
import math 

In [81]:
def get_frac_bits(
    in_float: float, 
    round_factor: int=8, #number of decimal places to round to. 
):
    decimal_part, int_part = np.modf(in_float)
    decimal_part = np.abs(np.round(decimal_part, round_factor))
    dec_string = str(decimal_part)[2:] #omit the '0.'

    if decimal_part == 0: 
        frac_bits = 0 
    else: 
        power = len(dec_string) 
        frac_bits = math.log(10**power, 2) #base 2 log. 
        frac_bits = int(frac_bits) + 1 #round up 

    return frac_bits  

In [120]:
"""
in hardware, if the number of bits for the integer part is not enough but there's the right
number for the decimal, the computer doesn't care; it just sees the binary string and will 
insert the decimal point according to the fixed precision you give it. Thus, if you have 
something like ap_fixed<4,2> for 6.25, even though the decimal part *could* be represented accurately 
with 2 bits, it won't because the integer part will overflow into it. That's why you should just concatenate 
the strings here. 
"""

#float to unsigned binary. Returns a list in the form of [unsigned binary representation, sign]
def float_to_ubin(
    in_float: float,
    round_factor: int=8, 
    round_num: bool=True
): 

    if in_float < 0: #store the sign. 
        sign = "-" 
    elif in_float > 0: 
        sign = "+" #me being explicit and also consistent 
    else: 
        sign = " "

    in_float = np.abs(in_float) 
    decimal_part, int_part = np.modf(in_float) 
    print(f"debugging -> int: {int_part}, decimal: {decimal_part}")
    
    if round_num: 
    #round the float to the specified decimal place 
        decimal_part = np.round(decimal_part, round_factor)
        print(decimal_part)
        in_float = int_part + decimal_part

    #from https://www.geeksforgeeks.org/python/python-program-to-convert-floating-to-binary/: 
    #IEEE 754 binary representation
    #binary_rep = np.binary_repr(np.float32(in_float).view(np.int32), width=32)
    #This is precise, but not exactly ideal for this particular context.  

    frac_bits = get_frac_bits(in_float) 
    print("Frac bits:", frac_bits) 
    scaled = int(in_float * (2**frac_bits)) 
    binary_string = format(scaled, 'b') #formats the scaled number as binary 

    #split the binary string into the integer and fractional portions. 
    #bin_int_length = len(bin(int(int_part))[2:]) #ignore the '0b'

    bin_list = [binary_string, sign] 
      
    return bin_list 

In [95]:
def twos_complement(
    bin_list: list
): 
    ubin_str, sign = bin_list 
    nbits = len(ubin_str) + 1

    #if width < len(ubin_str): 
    #    raise ValueError("Specified width not large enough.")

    if sign == '+': 
        sbin_str = '0' + ubin_str
         
    elif sign == '-': 
        val = (1 << nbits) + (-int(ubin_str, 2))
        sbin_str = format(val, f'0{nbits}b')
        
    else: 
        sbin_str = ubin_str #cuz it's just all zeros. 

    return sbin_str

#remember THE FIRST BIT IS THE SIGN BIT! We've gone over this before (idea where we need an extra bit for the sign.) 
  

In [121]:
#2s complement function should be called before this one. 
def fit_to_width(
    bin_str: str,  
    int_bits: int, 
    frac_bits: int
): 
    
    target_width = int_bits + frac_bits 
    current_width = len(bin_str) 

    if int_bits < 0 or frac_bits < 0: 
        raise ValueError("Number of bits cannot be negative!")

    #if current width is too small, right-pad with zeroes 
    if current_width < target_width: 
        for i in range(target_width - current_width):
            bin_str = bin_str + '0'

    if frac_bits == 0: 
        full_num = bin_str + '.' 
    elif int_bits == 0: 
        full_num = '.' + bin_str 
    else:    
        int_part = bin_str[:int_bits] 
        frac_part = bin_str[int_bits:int_bits+frac_bits]
        full_num = int_part + '.' + frac_part 

    return full_num #signed binary representation split into int + frac form 

In [125]:
def bin_to_int(
    bin_str: str
): 
    if '.' in bin_str: 
        bin_str = bin_str.replace('.', "")

    int_rep = int(bin_str, 2)

    return int_rep 

In [131]:
num = 5.79980469 
int_bits = 4
frac_bits = 12

print("Original float: ", num) 
ubin_num = float_to_ubin(num) 
print("Unsigned binary + sign: ", ubin_num) 

twocomp_num = twos_complement(ubin_num)
print("Twos complement: ", twocomp_num)

fixed_rep = fit_to_width(twocomp_num, int_bits, frac_bits)
print("Adjusted to specified ap_fixed width: ", fixed_rep)

int_rep = bin_to_int(fixed_rep) 
print("Final integer representation: ", int_rep)

Original float:  5.79980469
debugging -> int: 5.0, decimal: 0.7998046900000002
0.79980469
Frac bits: 27
Unsigned binary + sign:  ['101110011001100000000000000000', '+']
Twos complement:  0101110011001100000000000000000
Adjusted to specified ap_fixed width:  0101.110011001100
Final integer representation:  23756
